# Assignment#2- Hidden Markov Model 

In [1]:
import numpy as np
from hw2_fns import hmm_gen, Viterbi, match, Forward

In [2]:
# from the question
S = ['H', 'T']
T = 3
a = np.array([[  0,  .5,  .5], #transition probabilities
              [.01, .94, .05],
              [.01, .05, .94]])
e = np.array([[.5, .5], #emission probabilities
              [.8, .2]])

## 1. HMM Generator

In [3]:
# Example 
pi,x = hmm_gen(S, T, a, e)
print(f'length of generated sequence = {len(x)}')
print(f'Hidden state seq: \n {pi}')
print(f'Observable state seq: \n {x}')

length of generated sequence = 45
Hidden state seq: 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0]
Observable state seq: 
 ['T', 'T', 'T', 'T', 'H', 'H', 'H', 'H', 'H', 'T', 'H', 'T', 'T', 'T', 'H', 'T', 'H', 'T', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'T', 'H', 'H', 'T', 'T', 'T', 'H', 'T', 'H', 'H', 'H', 'H', 'T', 'H', 'H', 'H', 'H', 'H']


## 2. Viterbi Algorithm

In [4]:
# Example from the question
path = Viterbi(x, S, T, a, e)
matchfrac = match(path, pi)
print(f'{matchfrac*100:.2f}% of the inferred seq matches with the original hidden state sequence.')



90.91% of the inferred seq matches with the original hidden state sequence.


## 3. Forward

In [9]:
from hw2_fns import Forward
likelihood = Forward(x, S, T, a, e)
print(f'The likelihood of input sequence = {likelihood}') 

The likelihood of input sequence = 4.623937530261553e-15


## 4. Backward 

In [ ]:
T = np.arange(T)
obsstates = {S[0]: 0, S[1] : 1}
alpha = np.zeros((len(x), len(T)-1))
alpha[:, 0] = 0
alpha[0, :] = a[0, 1:]
path = []
for xi in range(1,len(x)):
    for hi in range(len(T)-1):
        k = []
        for hi2 in range(len(T)-1):
            p_trans = a[hi, hi2]
            p_emssn = e[hi-1, obsstates[x[xi]]]
            k.append(alpha[xi-1, hi2]*p_trans*p_emssn)
        alpha[xi, hi] = np.sum(k)
likelihood =  np.sum(alpha[-1, :])
return likelihood